**Función:** entrenamiento de arquitectura CNN alternativa (Kaggle, etc.), con el dataset 10k desbalanceado.

**Requiere:**
- ../../datasets/DATASET10K/ (con una carpeta por label y las fotos sin recortar)

**Crea:** Fichero con el modelo entrenado (epoch con menor val_loss).
- modelos/tf_model_\<run_id\>

In [ ]:
# !pip install mlflow
import mlflow
from tensorflow.random import set_seed
from numpy.random import seed
from tensorflow import keras
from tensorflow.keras.utils import image_dataset_from_directory
from keras.layers import Dense, Dropout, Activation, Flatten, Convolution2D, Conv2D, MaxPooling2D
from keras.models import Sequential
#from keras.optimizers import Nadam
from keras.callbacks import EarlyStopping
from keras.layers import Rescaling
from tensorflow.data import AUTOTUNE
from tensorflow import one_hot
from keras import backend as K
import pickle

In [ ]:
ejecutadoEnGoogleDrive = True # True si se ejecuta en Google Drive

In [4]:
if ejecutadoEnGoogleDrive:
    from google.colab import drive
    drive.mount('/content/drive')
    PATH = '/content/drive/clasificacion_imagenes/'
    ruta_dataset = '/content/drive/clasificacion_imagenes/datasets/DATASET10K/'
else:
    PATH = '../../1_Setup_MLOps'
    ruta_dataset = '../../datasets/DATASET10K/'

%cd {PATH}

Mounted at /content/drive


In [9]:
IMG_SIZE = 224
epochs = 100 # máximo de epochs (si no dispara el early stopping)
validation_split = 0.2
batch_size=64

# Fijar random state para obtener resultados reproductibles
seed(42) # So long and thanks for all the fish
set_seed(42)

# Definir experimento y run (MLflow)

In [7]:
# ir a la carpeta raiz del proyecto (donde se encuentra mlruns)
%cd 'drive/Shareddrives/The Valley - TFM'

/content/drive/Shareddrives/The Valley - TFM


In [8]:
# Este run pertenence al experimento 4, que tiene el id "3" (string)
experiment_id = "3"
mlflow.set_experiment(experiment_id = experiment_id) 
run = mlflow.start_run( # crear run en MLflow
    experiment_id=experiment_id,
    tags={"Autor": 'Toni_V', "Computer": "Colab", "params":"arq exp1 - 64 batch"}
)
# activar auto logging
mlflow.tensorflow.autolog()
print('Vamos empezar el run {} en el experimento {}. Este run está {}.'.format(run.info.run_id, experiment_id, run.info.status))

Vamos empezar el run 1741e2a698b84fbcaac36a4b33ea44c0 en el experimento 3. Este run está RUNNING.


# Definir carga del dataset y preprocesado

In [11]:
train_ds = image_dataset_from_directory(
  ruta_dataset,
  validation_split=validation_split,
  subset="training",
  seed=42,
  image_size=(IMG_SIZE,IMG_SIZE),
  batch_size=batch_size)\
.cache()\
.prefetch(buffer_size=AUTOTUNE).map(lambda x, y:
                                    (Rescaling(1./255)(x),
                                     one_hot(y, 5)))

Found 8312 files belonging to 5 classes.
Using 6650 files for training.


In [12]:
test_ds = image_dataset_from_directory(
  ruta_dataset,
  validation_split=validation_split,
  subset="validation",
  seed=42,
  image_size=(IMG_SIZE,IMG_SIZE),
  batch_size=batch_size)\
.cache()\
.prefetch(buffer_size=AUTOTUNE).map(lambda x, y:
                                    (Rescaling(1./255)(x),
                                     one_hot(y, 5)))

Found 8312 files belonging to 5 classes.
Using 1662 files for validation.


In [13]:
categories = ['drink', 'food', 'inside', 'menu', 'outside'] # train_ds.class_names
num_labels = len(categories)

In [14]:
model = Sequential()
model.add(Conv2D(10, (12, 12), padding='same', input_shape=(IMG_SIZE,IMG_SIZE,3)))
model.add(Activation('relu'))
model.add(Conv2D(IMG_SIZE, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(4, 4)))
model.add(Dropout(0.25))

model.add(Conv2D(IMG_SIZE, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(IMG_SIZE, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(5))
model.add(Activation('softmax'))
opt = keras.optimizers.Nadam(lr=0.0001, schedule_decay=1e-6)


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/nadam.py:73: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Nadam, self).__init__(name, **kwargs)


In [ ]:
""" ORIGINAL ARCH.
model = Sequential()
model.add(Conv2D(params['param_A'],
                 (12, 12),
                 padding='same',
                 input_shape=(IMG_SIZE,IMG_SIZE,3)))
model.add(Activation('relu'))
model.add(Conv2D(params['param_A']+10,
                 (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=params['param_B'][0]))
model.add(Dropout(0.25))

model.add(Conv2D(params['param_A']+20,
                 (3, 3),
                 padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(params['param_A']+30,
                 (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=params['param_B'][1]))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_labels))
model.add(Activation('softmax'))
opt = keras.optimizers.Nadam(learning_rate=0.0001, schedule_decay=1e-6)

"""

In [ ]:
"""
# PRUEBA ARQUITECTURA  KAGGLE
model = Sequential()
model.add(Conv2D(32,
                 (3, 3),
                 activation='tanh',
                 padding='same',
                 input_shape=(IMG_SIZE,IMG_SIZE,3)))

model.add(Conv2D(30,
                 (3, 3),
                 activation='tanh',
                 padding='same'))

model.add(MaxPooling2D(2, 2))

model.add(Conv2D(30,
                 (3, 3),
                 activation='tanh',
                 padding='same'))

model.add(MaxPooling2D(2, 2))

model.add(Conv2D(30,
                 (3, 3),
                 activation='tanh',
                 padding='same'))

model.add(Flatten())
model.add(Dense(20))
model.add(Activation('relu'))

model.add(Dense(14))
model.add(Activation('relu'))

model.add(Dense(num_labels))
model.add(Activation('softmax'))
opt = keras.optimizers.Nadam(learning_rate=0.0001, schedule_decay=1e-6)
"""

In [15]:
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [16]:
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['acc', f1_m, precision_m, recall_m])

In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 224, 224, 10)      4330      
                                                                 
 activation (Activation)     (None, 224, 224, 10)      0         
                                                                 
 conv2d_1 (Conv2D)           (None, 222, 222, 224)     20384     
                                                                 
 activation_1 (Activation)   (None, 222, 222, 224)     0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 55, 55, 224)      0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 55, 55, 224)       0         
                                                        

In [18]:
callback = EarlyStopping(monitor='val_loss',
                         patience=10,
                         restore_best_weights=True)

In [20]:
history = model.fit(train_ds,
                    validation_data=test_ds,
                    epochs = epochs,
                    batch_size=batch_size,
                    callbacks=[callback],
                    verbose=1)

Epoch 1/100
104/104 [==============================] - 817s 8s/step - loss: 0.9310 - acc: 0.6710 - f1_m: 0.6328 - precision_m: 0.7405 - recall_m: 0.5642 - val_loss: 0.8004 - val_acc: 0.7810 - val_f1_m: 0.6503 - val_precision_m: 0.8948 - val_recall_m: 0.5126
Epoch 2/100
104/104 [==============================] - 912s 9s/step - loss: 0.6980 - acc: 0.7617 - f1_m: 0.7461 - precision_m: 0.8113 - recall_m: 0.6928 - val_loss: 0.8892 - val_acc: 0.7034 - val_f1_m: 0.6217 - val_precision_m: 0.8581 - val_recall_m: 0.4892
Epoch 3/100
104/104 [==============================] - 918s 9s/step - loss: 0.6269 - acc: 0.7808 - f1_m: 0.7717 - precision_m: 0.8289 - recall_m: 0.7235 - val_loss: 0.7902 - val_acc: 0.7184 - val_f1_m: 0.6911 - val_precision_m: 0.8427 - val_recall_m: 0.5872
Epoch 4/100
104/104 [==============================] - 920s 9s/step - loss: 0.5690 - acc: 0.8018 - f1_m: 0.7921 - precision_m: 0.8447 - recall_m: 0.7470 - val_loss: 0.6407 - val_acc: 0.7804 - val_f1_m: 0.7587 - val_precision_m

2022/09/27 00:38:38 WARNING mlflow.utils.requirements_utils: Found tensorflow version (2.8.2+zzzcolab20220719082949) contains a local version label (+zzzcolab20220719082949). MLflow logged a pip requirement for this package as 'tensorflow==2.8.2' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2022/09/27 00:38:47 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmptbk41syf/model, flavor: keras), fall back to return ['tensorflow==2.8.2', 'keras==2.8.0']. Set logging level to DEBUG to see the full traceback.


# Guardar modelo entrenado

In [25]:
model.save('../4_Modelacion/Experimento_4/modelos/tf_model_exp1_{}'.format(run.info.run_id))

In [26]:
mlflow.end_run()